In [1]:
from dotenv import load_dotenv
load_dotenv()
import numpy as np
import pandas as pd
import os
import glob
import shutil

### copy the images from the THINGS database that were used in the fmri experiment (8740 of them) into its own directory.

In [2]:
dataset_root = os.path.join(os.getenv("DATASETS_ROOT", "/default/path/to/datasets"),"THINGS_fmri") #use default if DATASETS_ROOT env variable is not set.
save_root = os.path.join(dataset_root, "derivatives", "stimuli_metadata", "experimental_images")
if not os.path.exists(save_root):
    os.makedirs(save_root)

In [12]:
def split_filename(filename):
    # Remove the file extension
    base_name = os.path.splitext(filename)[0]
    
    # Split based on underscores
    parts = base_name.split('_')
    
    if len(parts) > 1:
        # Return everything except the last part
        return '_'.join(parts[:-1])
    else:
        # If there's no underscore or only one part, return the base name
        return base_name

In [ ]:
experimental_images = set()
for subject in ['01','02','03']:
    fmri_path = os.path.join(dataset_root,"derivatives", "GLM")
    task = 'things' 
    subject = f"sub-{int(subject):02}"
    session_paths = glob.glob(os.path.join(fmri_path, subject, f'ses-{task}*'))
    assert(len(session_paths) > 0)
    numsessions = len(session_paths)

    train_stimuli = []
    test_stimuli = []

    for ses in range(1,numsessions+1):
        session = f"ses-{task}{ses:02}"
        numruns = len(glob.glob(os.path.join(dataset_root, "derivatives", "fmriprep", subject, session, "func", f"{subject}_{session}_task-{task}_run-*_desc-confounds_timeseries.tsv"))) 
        for run in range(1,numruns+1):
            tmp = pd.read_table(os.path.join(dataset_root, "Nifti", subject, session, "func", f"{subject}_{session}_task-{task}_run-{run:02}_events.tsv"))
            for idx, img_path in enumerate(tmp.loc[:,'file_path']):
                if 'catch' in img_path:
                    continue
                filename = img_path.split('/')[-1]
                folder = split_filename(filename) #there seems to be an error where the img_filename 'coat/coat_rack_13s.jpg' doesnt exist. It should be 'coat_rack/coat_rack_13s.jpg'. The folder 'coat' exists too.
                if filename not in experimental_images:
                    experimental_images.add(filename)
                    if not os.path.isfile(os.path.join(save_root, filename)):
                        shutil.copy(os.path.join(dataset_root, "derivatives", "stimuli_metadata", "images", folder, filename), save_root)
    